# 3DX to IFS file

This script reads the latest 3dx files and extracts the columns we've configured for IFS migration.  This is a stop-gap while we are still working with extract files and will be replaced by a direct migration of data from 3DX straight into IFS - using IFS connect to drop files for processing on an internal IFS queue.


Requirements:    
An 3dx extracted csv file in Engineering BoM sharepoint directory for the selected project, eg 'T33-BoM-XP_collated_BOM.csv'


Inputs:
Project name (T50, T50s, T33_XP...)
Previous timestamp 

Outputs:   
Writes compare files, Delta files, and migration txt files to:   

Output dir = sharepoint dir, project, IFS   


In [1]:
import pandas as pd
import numpy as np
import os
import re
import io

import openpyxl
import excel_formatting
import logging
import argparse
import configparser

In [2]:
def type_of_script():
    '''
        determine where this script is running
        return either jupyter, ipython, terminal
    '''
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'

In [ ]:
def add_function_group(df):
    # Add Function and Sub Group if it doesn't already exist

    # - Level 0 = Model Variant   
    # - Level 1 = Function Group Area   
    # - Level 2 = System   
    # - Level 3 = Sub Systems
    # - level 4 = AMs/SAs??

    # Find each one and forward fill to the next occurrence
    # function group - level 1
    df['Function Group'] = np.where(df['Level'].isin([0,1]), df['Description'], None)
    df['Function Group'] = df['Function Group'].ffill()

    # System - level 2
    df['System'] = np.where(df['Level'] == 2, df['Description'], None)
    df['System'] = np.where(df['Level'] >= 2, df['System'].ffill(), None)

    # SUB_System = level 3
    df['Sub System'] = np.where(df['Level'] == 3, df['Description'], None)
    df['Sub System'] = np.where(df['Level'] >= 3, df['Sub System'].ffill(), df['Sub System'])

    return df

In [3]:
def update_parent_part(BOM):
    # reset index before trying to update, otherwise multiple rows get updated
    BOM.reset_index(inplace=True, drop=True)

    for sgroup, frame in BOM[BOM['Part Level'] > 2].groupby('Sub Group'):
        level = {}

        previous_parent_part=0

        for i, row in frame.iterrows():
            current_part_number = row['Part Number']
            current_part_level = row['Part Level']
            # reset higher levels for each assembly
            if current_part_level == 5:
                # remove entries from higher levels
                keys = [k for k in level if k > 5]
                for x in keys:
                    del level[x]

            # write part number to dictionary under current part level
            level[current_part_level] = current_part_number
            # update the current_parent_part if we have current part details (info from catia)
            # as we've created level 1 and 2 we don't need this check
            # print (current_part_level, current_part_number)
            level[2] = group_area_dict[sgroup]
            if i > 0:
            # get the max part level from the level dictionary that's less than current part level
                previous_parent_level = max(k for k in level if k < current_part_level)

                    # update the parent part
                # print (i, "Parent part {} from previous level {}".format(level[previous_parent_level], previous_parent_level))
                BOM.at[i,'Parent Part'] = level[previous_parent_level]
    return BOM

In [4]:
def db_pool_connection(env):
    # connection to database using sqlalchemy
    import oracledb
    from sqlalchemy import create_engine
    from sqlalchemy import text
    import db_config
    import pandas as pd

    # d = r"C:\Users\mark.chinnock\oracle\instantclient_21_10"
    # oracledb.init_oracle_client(lib_dir=d)

    if env == 'PREPROD':
        pool = oracledb.create_pool(user=db_config.user, password=db_config.PREPROD_userpwd, dsn=db_config.PREPROD_connect_string,
                                min=1, max=5, increment=1)
    elif env == 'LIVE':
        pool = oracledb.create_pool(user=db_config.user, password=db_config.LIVE_userpwd, dsn=db_config.LIVE_connect_string,
                                min=1, max=5, increment=1)
    elif env == 'Sandbox':
        pool = oracledb.create_pool(user=db_config.user, password=db_config.Sandbox_userpwd, dsn=db_config.Sandbox_connect_string,
                                min=1, max=5, increment=1)


    return pool

In [5]:
def get_ifs_part_cat(env):
    # connection to database using sqlalchemy
    import oracledb
    from sqlalchemy import create_engine
    from sqlalchemy import text
    import db_config
    import pandas as pd
    from contextlib import suppress

    # Database Credentials
    username = db_config.user
    password = db_config.LIVE_userpwd

    engine = create_engine(
        f'oracle+oracledb://:@',
            thick_mode=None,
            connect_args={
                "user": db_config.user,
                "password": db_config.LIVE_userpwd,
                "host": db_config.LIVE_host,
                "port": 1521,
                "service_name": db_config.LIVE_service
        })

    query = ("select distinct c.part_no, c.unit_meas, p.lot_tracking_code, p.serial_tracking_code, p.serial_rule "
            "from ifsapp.inventory_part c "
            "left join ifsapp.part_catalog p "
            "on c.part_no = p.part_no "
            "where c.part_no like 'T%'")

    try:
        with engine.connect() as connection:
            # print(connection.scalar(text("""SELECT * from IFSAPP.purchase_order_line_all""")))
            query = connection.execute(text(query))

        df = pd.DataFrame(query.fetchall())

        df.columns = df.columns.str.upper()


    except:
        pass

    

    return df


In [6]:
def create_group_area_dict(project):
    group_area_dict = {
        'A02-Panels & Closure Systems':project +'-01', 
        'A03-Exterior Systems':project +'-01',
        'A01-Structure Systems':project +'-01', 
        'B01-Suspension Systems':project +'-02',
        'C01-Braking Systems':project +'-02', 
        'D01-Steering Systems':project +'-02', 
        'E01-Pedal System':project +'-02',
        'M01-Control Systems':project +'-03', 
        'M02-Traction Systems':project +'-03',
        'M03-Electrical Distribution Sys':project +'-03', 
        'M04-Multimedia Systems':project +'-03',
        'M05-Safety & Security Systems':project +'-03', 
        'M06-Software Systems':project +'-03',
        'N01-Interior & Trim Systems':project +'-04', 
        'N02-HVAC Systems':project +'-04',
        'F01-ICE Powertrain Systems':project +'-03', 
        'G01-Transmission Systems':project +'-03',
        'J01-Pwt NVH & Heatshield Sys':project +'-03', 
        'L01-Cooling Systems':project +'-03',
        'R01-Styling':project +'-07',
        'P01-Packaging':project +'-06',
        'T01-Tooling':project +'-08',
        'U01-Development':project +'-09',
        'V01-Accessories':project +'-10'
    }

    return group_area_dict

In [7]:
def create_part_type_dict():
    part_type_dict = {
    'AIH':'Manufactured',
    'BOF':'Purchased',
    'BOP':'Purchased',
    'CON':'Purchased (Raw)',
    'ENG':'Purchased (Raw)',
    'FAS':'Purchased (Raw)',
    'FIP':'Purchased (Raw)',
    'RAW':'Purchased (Raw)',
    'MIH':'Manufactured',
    'POA':'Manufactured',
    'MOB':'Manufactured'
    }

    return part_type_dict


In [8]:
def find_sharepoint_dir(project):
    import glob
    # personal one drive
    user_dir = 'C:/Users/USERNAME'

    if project in (['T33_XP','T50s']):
        company = 'gmd'
    else:
        company = 'gmt'
        CMO = True

    # replace USERNAME with current logged on user
    user_dir = user_dir.replace('USERNAME', os.getlogin())
    # find out what gordonmurray folder to use from the user_dict

    # read in config file
    config = configparser.ConfigParser()
    config.read('user_directory.ini')

    # read in gm_dir and gm_docs from config file
    gm_dir = config[os.getlogin().lower()]['gm_dir']
    gm_docs = config[os.getlogin().lower()][company]

    # go find the Engineering BoM directory within the User directory
    # from glob import glob
    sharepoint = user_dir + "/" + gm_dir + "/" + gm_docs

    return (sharepoint, company, CMO)

In [9]:
def open_existing_bom(base, bom_file):
    path = os.path.join(base, bom_file)
    with open(path, "rb") as f:
        existing_bom = pd.read_excel(f, na_values='*', parse_dates=True) 
        # sheetnames = [sheet for sheet in f.sheet_names]

    return existing_bom

In [10]:
def clear_not_set_values(df):
    df = df.replace('Not Set', np.NaN)

    return df

In [11]:
def create_parent_part(df):
    # reset index before trying to update, otherwise multiple rows get updated
    df.reset_index(inplace=True, drop=True)
    df['Parent Part'] = None

    level = {}
    previous_parent_part=0

    for i, row in df.iterrows():
        current_part_number = row['Title']
        current_part_level = row['Level']

        # write part number to dictionary under current part level
        level[current_part_level] = current_part_number

        # reset higher levels for each assembly
        # remove entries from higher levels
        keys = [k for k in level if k > current_part_level]
        for x in keys:
            del level[x]

        if current_part_level > 0:
            # get the max part level from the level dictionary that's less than current part level
            previous_parent_level = max(k for k in level if k < current_part_level)

            # update the parent part
            # print (i, "Parent part {} from previous level {}".format(level[previous_parent_level], previous_parent_level))
            df.at[i,'Parent Part'] = level[previous_parent_level]
    return df

In [12]:
def create_sa_index(df):
    # leave this as int and you get the index with .0 at the end, which helps match row 5, but not row 50, 500, etc
    df['SA_Index'] = np.where(df['Level'] == 4, df['orig_sort'].astype(str), np.nan)
    df['SA_Index'] = np.where(df['Level'] < 4, df['orig_sort'].astype(str), df['SA_Index'])
    # forward fill so that > Level 5 get the same index
    df['SA_Index'] = df['SA_Index'].ffill()
    # don't include Part level in SA_Index
    # cleansed_df['SA_Index'] = cleansed_df['SA_Index'].astype(str) + '_' + cleansed_df['Part Level'].astype(str) + '_' + cleansed_df['Part Number']
    # df['SA_Index'] = df['SA_Index'].astype(str) + '_' + df['Title']
    return df

In [13]:
def create_sa_index2(df):
    df['SA_Index'] = np.where(df['Assembly'], df['orig_sort'].astype(str), np.nan)
    df['SA_Index'] = df['SA_Index'].ffill()

    return df

In [14]:
def correct_FIPS(df):
    df['BOF_Parent'] = np.where(df['Source Code'] == 'BOF', df['Title'], np.nan)
    df['BOF_Parent'].ffill(inplace=True)
    df['Parent Part'] = np.where(df['Source Code'] == 'FIP', df['BOF_Parent'], df['Parent Part'])

    return df

In [15]:
def read_prev_mig_files(outdir, prev_timestamps):
    import glob
    # /**/ makes this recursive through folders in the project specfied
    previous_struct_file = glob.glob(outdir + '/**/Structure_*' + prev_timestamps + '.txt', recursive = True)[0]
    previous_part_file = glob.glob(outdir + '/**/Part*' + prev_timestamps + '.txt', recursive = True)[0]

    # use dtype in read_csv to capture trailing zeros in ENG_PART_REV
    path = os.path.join(base, 'IFS', previous_part_file)
    with open(path, "rb") as f:
        prev_part = pd.read_csv(f, sep='\t', dtype={'ENG_PART_REV':str}) 
    path = os.path.join(base, 'IFS', previous_struct_file)
    with open(path, "rb") as f:
        prev_struct = pd.read_csv(f, sep='\t', dtype={'SUB_PART_REV':str}) 
        # sheetnames = [sheet for sheet in f.sheet_names]

    return prev_part, prev_struct

In [16]:
def create_structure_parts(cleansed_df):
# only create structure_df with 4 cols, using Revision once - otherwise we can't rename the columns without it renaming both Revision cols
# we'll add the PART_REV later by mapping to the parent_part_dict

    structure_cols = [
    'Parent Part',
    'Revision',
    'Title',
    'Quantity',
    ]

    # bring in Release Status from BoM as well
    part_cols = [
    'Title',
    'Level',
    'Description',
    'Weight',
    'Source Code',
    'Revision',
    'Function Group',
    'Sub System',
    'Owner',
    'Maturity State',
    'UOM',
    'Provide',
    'Inventory Part Planning',
    # bring in Phase to be mapped to DEVELOPMENT PHASE
    'Phase'
    ]

    # don't need to bring the first row from cleansed_df into the structure file.
    structure_df = cleansed_df[1:][structure_cols].copy()
    parts_df = cleansed_df[part_cols].copy()

    # rename cols to match IFS naming
    structure_df.rename(columns={
        'Parent Part':'PART_NO', 
        'Title':'SUB_PART_NO',
        'Quantity':'QTY',
        'Revision':'SUB_PART_REV'   
    }, inplace=True)

    parts_df.rename(columns={
        'Title':'PART_NO',
        'Description':'DESCRIPTION',
        'Weight':'WEIGHT_NET',
        'Function Group':'FUNCTION_GROUP',
        'Sub System':'SUB_GROUP',
        'Level':'PART_LEVEL',
        'Owner':'PART_RESPONSIBLE',
        'Source Code':'SOURCE_CODE',
        'Maturity State':'RELEASE_STATUS',
        'UOM':'UNIT_CODE',
        'Provide':'PROVIDE',
        'Inventory Part Planning':'INVENTORY_PART_PLANNING',
        # mapping Phase to MATURITY until Ryan made his changes
        'Phase':'MATURITY'
        }
        , inplace=True)


    return structure_df, parts_df

In [17]:
def create_IFS(df):

    cols = ['orig_sort',
    'Function Group',
    'System',
    'Sub System', 
    'Level',
    'Title', 
    'Revision',
    'Description', 
    'Quantity',
    'Source Code',
    'UOM',
    'Provide',
    'Actual Mass',
    'CAD Mass',
    'CAD Maturity',
    # 'Change Type',
    'Maturity State',
    'Inventory Part Planning',
    'Name',
    'Owner',
    # Part Type from 3dx is different to what IFS Part type is
    # 'Part Type',
    # 'Piece Cost',
    # 'Piece Cost Estimated',
    # 'Piece Cost Maturity',
    # 'Service Part Flag',
    # 'Tooling Cost Target',
    # 'Tooling Cost Estimated',
    # 'Assembly',
    # 'Part',
    'Phase'
    ]

    # merged = pd.merge(df, existing_bom[cols], on='Part Number', how='left', indicator=True)
    IFS = df[cols]

    return IFS

In [18]:
def build_delta_files(parts_df, structure_df):
    # build a dictionary from a copy
    dict_part_compare = {1:prev_part.copy(),2:parts_df.copy()}
    dict_struct_compare = {1:prev_struct.copy(),2:structure_df.copy()}

    dict_part_compare[1]['ENG_PART_REV'] = dict_part_compare[1]['ENG_PART_REV'].astype(str).str.split('.').str[0]
    dict_part_compare[2]['ENG_PART_REV'] = dict_part_compare[2]['ENG_PART_REV'].astype(str).str.split('.').str[0]
    dict_struct_compare[1]['PART_REV'] = dict_struct_compare[1]['PART_REV'].astype(str).str.split('.').str[0]
    dict_struct_compare[2]['PART_REV'] = dict_struct_compare[2]['PART_REV'].astype(str).str.split('.').str[0]
    dict_struct_compare[1]['SUB_PART_REV'] = dict_struct_compare[1]['SUB_PART_REV'].astype(str).str.split('.').str[0]
    dict_struct_compare[2]['SUB_PART_REV'] = dict_struct_compare[2]['SUB_PART_REV'].astype(str).str.split('.').str[0]    

    dict_struct_compare2=pd.concat(dict_struct_compare)
    dict_part_compare2=pd.concat(dict_part_compare)

    # ignore_cols = ['WEIGHT_NET']
    dict_part_compare2.WEIGHT_NET = np.round(dict_part_compare2.WEIGHT_NET,4).astype(str)

    subset_cols = []
    if ignore_cols_for_comparison is not None:
        print ("cols being ignored {}".format(ignore_cols_for_comparison))
        subset_cols = dict_part_compare2.drop(columns=ignore_cols_for_comparison).columns
        # delta_parts and delta_struct have the rows with changes
        delta_parts = dict_part_compare2[dict_part_compare2['PART_LEVEL']>=4].drop_duplicates(subset=subset_cols, keep=False)

    else:
        # print ("no cols to ignore")
        delta_parts = dict_part_compare2[dict_part_compare2['PART_LEVEL']>=4].drop_duplicates(keep=False)

    delta_struct = dict_struct_compare2.drop_duplicates(keep=False)

    return delta_parts, delta_struct

In [19]:
def check_make_no_buy(df):
    # if MAKE and there is no BUY below next row's part level less than or equal to current part level we have a MAKE without a BUY
    # df['PROVIDE'] = np.where(df['Source Code'].isin(['AIH','MIH','MOB']),'Make','Buy')
    make_no_buy = list(df[(df['Source Code'].isin(['AIH','MIH','MOB'])) & (df['Part Level'].shift(-1) <= df['Part Level'])].SA_Index)
    make_no_buy = sorted(make_no_buy)
    return make_no_buy

In [20]:
def highlight_diff(data, color='pink'):
    # Define html attribute
    attr = 'background-color: {}'.format(color)
    other = data.xs('Previous', axis='columns', level=-1)
    # Where data != other set attribute
    return pd.DataFrame(np.where((data.ne(other, level=0)), attr, ''),
                        index=data.index, columns=data.columns)    

In [21]:
def compare_parts(df):
    try:
        df_all = pd.concat([df.loc[1,].set_index('PART_NO'), df.loc[2,].set_index('PART_NO')], axis='columns', keys=['Previous','Current'])
        df_final = df_all.swaplevel(axis='columns')[delta_parts.columns[1:]].fillna('')
    except:
        df_all = pd.concat([pd.DataFrame('', columns=delta_parts.columns, index=delta_parts['PART_NO']), delta_parts.loc[2,].set_index('PART_NO')], axis='columns', keys=['Previous','Current'])
        df_final = df_all.swaplevel(axis='columns')[delta_parts.columns[1:]].fillna('')

    return df_final

In [22]:
def compare_struct(df):
    indx = ['PART_NO','SUB_PART_NO']
    df_all = pd.concat([df.loc[1,].set_index(indx), df.loc[2,].set_index(indx)], axis='columns', keys=['Previous','Current'])
    df_final = df_all.swaplevel(axis='columns')[df.drop(columns=indx).columns].fillna('')

    return df_final

In [23]:
def drop_missing_source_codes(df):
    # find blank source codes and drop the whole assembly
    empty_sc = set(df['SA_Index'][(df['Source Code'].isna()) & (df['Level'] > 3)])

    # remove any assemblies with empty source codes
    if len(empty_sc) > 0:
        pat = '|'.join(r"\b^{}\b".format(x) for x in empty_sc)
        sc_df = df[~df['SA_Index'].str.contains(pat)]

    else:
        logging.info("There are no blank source codes to worry about")
        sc_df = df

    return sc_df


In [24]:
def drop_missing_source_codes2(df):
    # attempting to drop rows for this part and any child part, but you need to iterate to the bottom of the assembly.
    # might as well drop the whole assembly
    empty_sc = set(df['Title'][(df['Source Code'].isna()) & (df['Level'] > 3)])

    # remove any row with a 'Title' or 'Parent Part' that matches empty_sc.
    if len(empty_sc) > 0:
        pat = '|'.join(r"\b^{}\b".format(x) for x in empty_sc)
        sc_df = df[~df['Title'].str.contains(pat, na=False)]
        sc_df = sc_df[~sc_df['Parent Part'].str.contains(pat, na=False)]


    else:
        logging.info("There are no blank source codes to worry about")
        sc_df = df

    return sc_df

In [25]:
def drop_unreleased_assy(df):
    # find unreleased parts/assemblies.  We can't pass any assembly to IFS that isn't completely released.
    # must check there is something in the sa_set

    # set to give us a unique list
    sa_set = set(df['SA_Index'])

    if len(sa_set) > 0:
        unrel_sa_set = set(df['SA_Index'][df['Maturity State'] != 'Released'])

    # get the fully released assemblies by ignoring the ones containing unreleased sa_index
    # must check there is something in the unrel_sa_set
    if len(unrel_sa_set) > 0:
        pat = '|'.join(r"\b^{}\b".format(x) for x in unrel_sa_set)
        rel_df = df[~df['SA_Index'].str.contains(pat)]
        # get the unrel rows for writing out the warning messages
        unrel_df = df[df['SA_Index'].str.contains(pat)]
    else:
        # there are no non released sa to worry about.  This will return an empty df
        logit.info("There are no unreleased parts or assemblies to worry about")
        rel_df = df

    return rel_df


In [26]:
def handle_CMO(df):
    # forseven / GMT are not going to manufacture any parts themselves but send to a customer
    # therefore, there are no buy assemblies, but these should be phantoms to allow IFS to send parts to an external customer
    # input: pass in the parts file 
    # action: change all level 4 (assemblies) to 'Phantom' - leave everything else the same
    # output: return the parts file with the new Provide value of 'Phantom'

    df['Provide'] = np.where(df['Level'] == 4, 'Phantom', df['Provide'])

    return df

In [27]:
def get_current_time():
    import time
    timestr = time.strftime("%Y%m%d-%H%M")

    return timestr


# Main processing

In [28]:
if type_of_script() == 'terminal':
    parser = argparse.ArgumentParser()
    parser.add_argument("project", metavar='Project', type=str, help="T50, T50s, T33_XP, etc") 
    parser.add_argument("bom_file", metavar="BoM File", type=str, help="Updated_T48e-01-Z00001_2023-08-18.xlsx")
    parser.add_argument("timestamps", metavar='prev_timestamps', type=str, help='timestamp portion from previous bom file: eg 20230530-2044')
    parser.add_argument("env", metavar='Environment', type=str, help='LIVE, PREPROD or Sandbox')
    # parser.add_argument("incrementer", metavar='Incrementer', type=int)    
    # parser.add_argument('-d', help='Produce Delta files', action='store_true')
    parser.add_argument('-i', '--ignore', action='append', help="Cols to Ignore from comparison - only needed when we've added/removed a column from migration files")

    args = parser.parse_args()
    project = args.project
    bom_file = args.bom_file
    prev_timestamps = args.timestamps
    env=args.env
    ignore_cols_for_comparison = args.ignore
    # incrementer=args.incrementer

else:
    # set defaults if we're running in jupyter
    project = 'T53'
    # for producing the delta files
    bom_file = "Updated_T53-Z00002_2024-02-20.xlsx"
    prev_timestamps = 'None'
    env = 'LIVE'
    ignore_cols_for_comparison = None
    # incrementer=6
    # env = 'Sandbox'

  
print ("ignore cols: {}".format(ignore_cols_for_comparison))
print ("project: {}".format(project))
print ("bom file: {}".format(bom_file))
print ("env: {}".format(env))

sharepoint_dir, company, CMO = find_sharepoint_dir(project)

print ("company: {}".format(company))
print ("CMO: {}".format(CMO))

base = os.path.join(sharepoint_dir, project)
# base = user_dir

# where we'll write out the files
outdir = os.path.join(base, 'IFS')

timestr = get_current_time()
# logfile name and location
logfile = os.path.join(base, 'IFS', 'logs', "BOM_to_IFS_{}_{}_log.txt".format(project, timestr))
logging.getLogger('matplotlib').setLevel(logging.WARNING)
logging.basicConfig(filename=logfile, filemode='a', level=logging.DEBUG, format='%(asctime)s %(levelname)s:%(message)s')

logit = logging.getLogger(__name__)

logit.info("Starting the process...")
logit.info("Creating files for project {}".format(project))
logit.info("Running from {}".format(type_of_script()))
logit.info("base: {}".format(base))
logit.info("outdir: {}".format(outdir))
logit.info("Columns being ignore in comparison later (new cols?): {}".format(ignore_cols_for_comparison))

ignore cols: None
project: T53
bom file: Updated_T53-Z00002_2024-02-20.xlsx
env: LIVE
company: gmt
CMO: True


In [30]:
# open exsting bom
existing_bom = open_existing_bom(base, bom_file)

# Replace underscore in revision
Secondary Underscore revisioning was introduced into 3dx on 07/07/2023 and any pat created before this date will get an underscore revision format if it is uprevised/released.  Any part created afterwards will get the normal revision format.  This is a p in the a.  IFS will not accept underscore in the revision field.

Changing underscores to '.' - so revision 1_2.1 will become 1.1.1  This will work until we reach 7 characters (including the dots) as IFS can only handle 6 chars

Since 3dx migration to GMT tenant, underscores are removed from Revision.  Need to make revision field an object otherwise defaults to float64

In [36]:
existing_bom['Revision'] = existing_bom['Revision'].astype('str')

In [37]:
existing_bom['Revision'] = existing_bom['Revision'].str.replace('_', '.', regex=True)


In [38]:
# drop first row of percentage missing values
existing_bom = existing_bom.iloc[1:]

In [40]:
# clear 'not set' values as they are blank/not completed cells
existing_bom = clear_not_set_values(existing_bom)

In [41]:
# call IFS for latest parts catalogue info

# get db pool connection
pool = db_pool_connection(env)

query = ("select distinct c.part_no, c.unit_meas, p.lot_tracking_code, p.serial_tracking_code, p.serial_rule "
        "from ifsapp.inventory_part c "
        "left join ifsapp.part_catalog p "
        "on c.part_no = p.part_no "
        "where c.part_no like 'T%'")

with pool.acquire() as connection:
    ifs_parts_cat = pd.read_sql(query, connection)



# ifs_parts_cat = get_ifs_part_cat(env)



C:\Users\mark.chinnock\AppData\Local\Temp\ipykernel_32544\1709332845.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ifs_parts_cat = pd.read_sql(query, connection)


In [42]:
# parts_cat_dict for looking up serial tracking setting
parts_cat_dict = {}	
parts_cat_dict = pd.Series(ifs_parts_cat['LOT_TRACKING_CODE'].values,index=ifs_parts_cat['PART_NO']).to_dict()

ifs_parts_dict = {}
ifs_parts_dict = pd.Series(ifs_parts_cat['UNIT_MEAS'].values,index=ifs_parts_cat['PART_NO']).to_dict()    

ifs_serial_tracking = {}
ifs_serial_tracking = pd.Series(ifs_parts_cat['SERIAL_TRACKING_CODE'].values,index=ifs_parts_cat['PART_NO']).to_dict()


In [43]:
ifs_parts_cat[ifs_parts_cat['PART_NO'] == 'TFF-AB273']

,PART_NO,UNIT_MEAS,LOT_TRACKING_CODE,SERIAL_TRACKING_CODE,SERIAL_RULE
4194,TFF-AB273,pcs,Not Lot Tracking,Not Serial Tracking,Manual


In [44]:
# remove orig_sort if it's there
try:
    existing_bom.drop('orig_sort', axis=1, inplace=True)
except:
    pass

existing_bom.reset_index(inplace=True)
existing_bom.rename(columns={'index':'orig_sort'}, inplace=True)


In [45]:
IFS = create_IFS(existing_bom)

In [49]:
# have dropped part_type from 3dx and deriving settings for IFS from source code value
part_type_dict = create_part_type_dict()
IFS.loc[:,'Part Type'] = IFS['Source Code'].map(part_type_dict)


C:\Users\mark.chinnock\AppData\Local\Temp\ipykernel_32544\1918780858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IFS.loc[:,'Part Type'] = IFS['Source Code'].map(part_type_dict)


# Drop PACKAGING Function Group

In [52]:
# drop PACKAGING Function Group - all sorts happens in here and it is not required in ERP
IFS = IFS[~IFS['Function Group'].str.contains('PACKAGING')]


In [53]:
unstacked = IFS.groupby(['Title','Revision']).size().unstack()

# find number of columns dynamically, as number of unique status controls the number of columns
expected_revision_count = len(unstacked.columns) - 1

unstacked2 = unstacked[unstacked.isna().sum(axis=1)!=expected_revision_count].reset_index()
# dup_parts = unstacked2['Part Number'].tolist()

In [54]:
# where are they parts with more than one revision
unstacked2

Revision,Title,1.0,1.1,1.2,2.0,2.1,3.0,3.1,4.0,6.0,7.0,8.1
0,T53-P00637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0
1,T53-P01703,NaN,NaN,NaN,4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,T53-P01705,NaN,NaN,NaN,4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,T53-P02651,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


# Empty Structures

Lorena says if empty structure for 'BOF,'BOP', then it should be purchased (Raw) rather than purchased

Empty structure = following row has part level <= current part level

*** Even though this logic works, we don't supply Part Type to the migration process and Migration itself does this empty structure logic instead ***

In [55]:
# if BOF or BOP and next row's part level <= current part level update to Purchased (Raw)
#shift(-1) reads the next row
IFS['Part Type'] = np.where(IFS['Source Code'].isin(['BOF','BOP']) & (IFS['Level'].shift(-1) <= IFS['Level']), 'Purchased (Raw)', IFS['Part Type'])


# Drop Source Codes

03/03/2023: we are going to drop POA rows completely from the files and then correct the parent part to maintain the structure   
08/03/2023: we are dropping ENG rows completely from the dataframe and correct the parent part, if needed   
09/03/2023: we are dropping SOP rows as well now   
21/07/2023: we want to include SOP rows where there is a Service Identifier of 'Y' or 'C' and it is configured for a PROD vehicle


In [56]:
# drop source codes
# IFS = IFS[~IFS['Source Code'].isin(['POA','ENG'])]
IFS = IFS[~IFS['Source Code'].isin(['POA','ENG','SOP'])]

In [57]:
# Title (Part Number) cannot contain lowercase in IFS
IFS['Title'] = IFS['Title'].str.upper()

IFS_pp = create_parent_part(IFS)

# Correct FIPs 

FIP parent part needs to be the previous BOF - not sure if this is still valid for GMT


In [70]:
cleansed_df = correct_FIPS(IFS_pp)

In [71]:
cleansed_df['Weight'] = np.where(cleansed_df['Actual Mass'] > 0, cleansed_df['Actual Mass'], np.NaN)
cleansed_df['Weight'] = np.where((cleansed_df['Weight'].isna()) & (cleansed_df['CAD Mass'] > 0), cleansed_df['CAD Mass'], np.NaN)

# Create SA Index


In [72]:
# create_sa_index2 is using the assembly flag from 3dx
cleansed_df = create_sa_index(cleansed_df)

# Apply business rules

Drop unreleased assy   
Drop assy with make parts and no buy child parts   
Drop assy with missing mandatory data   
Drop part numbers > 25 chars long
Add released FIPS in unreleased assy to parts delta file   
Change Provide to Phantom if we are using a CMO for manufacturing

In [73]:
def populate_UOM(df):
    # # fill blank UNIT_CODE with LTR for FLA
    df.loc[:,'UOM'] = np.where(((df['UOM'].isna()) & (df['Source Code'] == 'FLA')), 'LTR', df['UOM'])

    # fill remaining blank UNIT_CODE with PCS
    df.loc[:,'UOM'] = df['UOM'].fillna('PCS')

    return df


In [74]:
def populate_Source_Code(df):
    # fill TFF parts with 'FAS' and populate Provide with 'Buy'
    # fill TPP parts with 'BOP' and populate Provide with 'Buy'
    df.loc[:,'Source Code'] = np.where(df['Title'].str.contains('^TFF', regex=True, na=False), 'FAS', df['Source Code'])
    df.loc[:,'Source Code'] = np.where(df['Title'].str.contains('^TPP', regex=True, na=False), 'BOP', df['Source Code'])
    df.loc[:,'Provide'] = np.where(df['Source Code'].isin(['FAS','BOP']), 'Buy', df['Provide'])

    return df

In [75]:
def populate_mandatory_fields(df):
    # just in case these fields haven't been populated in 3dx
    df.loc[:,'Provide'] = np.where((df['Provide'].isna() ) & (df['Source Code'].isin(['AIH','MIH','MOB'])),'Make','Buy')
    df.loc[:,'Inventory Part Planning'] = np.where((df['Inventory Part Planning'].isna() ) & ( df['Provide'] == 'Make'), 'P', 'A')

    return df

In [76]:
def long_part_numbers(df, length):
    long_pn = df[['Title','SA_Index']][df['Title'].str.len() > length]

    # remove any assemblies with long part numbers
    if len(long_pn) > 0:
        pat = '|'.join(r"\b^{}\b".format(x) for x in long_pn['SA_Index'])
        df_removed = df[~df['SA_Index'].str.contains(pat)]

        for x, row in long_pn.iterrows():
            logging.error("Part Number greater than {} chars: {}".format(length, row.iloc[0]))

    else:
        # nothing to remove - return the original df
        df_removed = df
    return (df_removed, long_pn)


In [77]:
# capitalise Source Code
cleansed_df.loc[:,'Source Code'] = cleansed_df['Source Code'].str.upper()

# for testing, populate UOM
cleansed_df = populate_UOM(cleansed_df)

# for testing, and perhaps for the long term populate TPP and TFF parts correctly with FAS and Buy
cleansed_df = populate_Source_Code(cleansed_df)

# in case some of the fields have been populated using the XEN tool in 3dx and some have been missed
cleansed_df = populate_mandatory_fields(cleansed_df)


# Drop blank source codes

In [78]:
# what if we don't do this for the time-being?  let everything get written out
# cleansed_df = drop_missing_source_codes(cleansed_df)

# Drop long part numbers and their assemblies

In [80]:
# drop assemblies with part numbers greater than 25 chars
cleansed_df, long_pn = long_part_numbers(cleansed_df, 25)

# CMO processing

If we are going to CMO cars and not manufacture them ourselves we don't need to Buy assemblies

In [81]:
# CMO is setup earlier when setting the company to GMT - wanted to use a separate flag in case gmt doesn't always mean CMO true
if CMO:
    cleansed_df = handle_CMO(cleansed_df)

C:\Users\mark.chinnock\AppData\Local\Temp\ipykernel_32544\1681251629.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Provide'] = np.where(df['Level'] == 4, 'Phantom', df['Provide'])


# Create structure and parts

In [82]:
structure_df, parts_df = create_structure_parts(cleansed_df)

# Populate missing values in the top levels
We need these rows for the original structure file, so we can't afford to drop them

In [83]:
# in case they haven't been populated in 3dx, we can fill level 0-3 with SC=SYS, PROVIDE=Make and INVENTORY_PART_PLANNING=P
parts_df['SOURCE_CODE'] = np.where((parts_df['SOURCE_CODE'].isna()) & (parts_df['PART_LEVEL'] < 4), 'SYS', parts_df['SOURCE_CODE'])
parts_df['PROVIDE'] = np.where((parts_df['PROVIDE'].isna()) & (parts_df['PART_LEVEL'] < 4), 'Make', parts_df['PROVIDE'])
parts_df['INVENTORY_PART_PLANNING'] = np.where((parts_df['INVENTORY_PART_PLANNING'].isna()) & (parts_df['PART_LEVEL'] < 4), 'P', parts_df['INVENTORY_PART_PLANNING'])


In [84]:
# find the issue level for the parent part.  Then map this to the PART_REV column of the structure_df

parent_part_dict = {}	
# parent_part_dict = pd.Series(IFS['Issue Level'][IFS['Part Number'].isna()==False].values,index=IFS['Part Number'][IFS['Part Number'].isna()==False]).to_dict()
# changed this to use cleansed_df
parent_part_dict = pd.Series(cleansed_df['Revision'][cleansed_df['Title'].isna()==False].values,index=cleansed_df['Title'][cleansed_df['Title'].isna()==False]).to_dict()

# this creates the PART_REV column
structure_df['PART_REV'] = structure_df['PART_NO'].map(parent_part_dict)

# now correct the column ordering for the final file template
structure_df = structure_df[['PART_NO', 'PART_REV', 'SUB_PART_NO', 'QTY', 'SUB_PART_REV']]



In [85]:
structure_df[structure_df['PART_NO'] == 'T48E-A02191']
parts_df[parts_df['PART_NO'] == 'T48E-A02191']

,PART_NO,PART_LEVEL,DESCRIPTION,WEIGHT_NET,SOURCE_CODE,Revision,FUNCTION_GROUP,SUB_GROUP,PART_RESPONSIBLE,RELEASE_STATUS,UNIT_CODE,PROVIDE,INVENTORY_PART_PLANNING,MATURITY


In [86]:
parts_df['INFO_TEXT'] = 'Drawing URL?'
# map LOT_TRACKING_CODE from existing parts, 22/06/2023 - changed to default to Not Lot Tracking.  This is usual default
parts_df['LOT_TRACKING_CODE'] = parts_df['PART_NO'].map(parts_cat_dict).fillna('Not Lot Tracking')
# parts_df['LOT_TRACKING_CODE'] = 'Order Based'
parts_df['SERIAL_RULE'] = 'Manual'
parts_df['CONFIGURABLE'] = 'Not Configured'
parts_df['AQUISITION_CODE'] = np.where(parts_df['PROVIDE'] == 'Phantom', 'No Demand', 'Demand')
parts_df['PLANNING_METHOD'] = np.where(company=='gmt','PMRP Planned', 'Standard Planned')
# lookup part issue level for ENG_REV_NO.
parts_df['ENG_PART_REV'] = parts_df['Revision']
# lookup if this is a parent part with a different issue level, otherwise leave as the part issue level
# parts_df['ENG_PART_REV'] = parts_df['PART_NO'].map(parent_part_dict).fillna(parts_df['ENG_PART_REV'])
# look up existing part information for serial tracking first
parts_df['SERIAL_TRACKING_CODE'] = parts_df['PART_NO'].map(ifs_serial_tracking)
# and then set all remaining isna() to Not Serial Tracker
parts_df['SERIAL_TRACKING_CODE'] = np.where(parts_df['SERIAL_TRACKING_CODE'].isna(), 'Not Serial Tracking', parts_df['SERIAL_TRACKING_CODE'])
# map to an existing UNIT_CODE if we have one, otherwise keep it as passed in. 
parts_df['UNIT_CODE'] = parts_df['PART_NO'].map(ifs_parts_dict).fillna(parts_df['UNIT_CODE'])
# part level 0 is exception to the rule above and will always be PROVIDE = 'Make', INVENTORY_PART_PLANNING = 'A'
parts_df['INVENTORY_PART_PLANNING'] = np.where(parts_df['PART_LEVEL'] == 0, 'A', parts_df['INVENTORY_PART_PLANNING'])
# create default status for inventory part status of 'A' for purchase/purchase raw, and 'I' for make parts
parts_df['INVENTORY_PART_STATUS'] = np.where(parts_df['PROVIDE']=='Make', 'A', 'I')

# don't need these columns anymore
parts_df.drop(columns=['Revision'], inplace=True)


In [87]:
# default weights to zero where not provided
parts_df['WEIGHT_NET'] = np.where(parts_df['WEIGHT_NET'].isna(), 0, parts_df['WEIGHT_NET'])

# get rid of negative weights whilst we're waiting for BoM to be corrected
parts_df['WEIGHT_NET'] = np.where(parts_df['WEIGHT_NET'] < 0, 0, parts_df['WEIGHT_NET'])

# add blank VARIANT but replacing MATURITY with DEVELOPMENT 
parts_df['VARIANT'] = np.NaN



In [88]:
# ENG_REV_NO is an IFS internally incremented number.  We don't have control over it so we are not going to pass it.
# migration script in IFS will handle this

part_cols_ordered = ['PART_NO',
'DESCRIPTION',
'WEIGHT_NET',
'INFO_TEXT',
'UNIT_CODE',
'LOT_TRACKING_CODE',
'SERIAL_RULE',
'SERIAL_TRACKING_CODE',
'CONFIGURABLE',
'PROVIDE',
'AQUISITION_CODE',
'PLANNING_METHOD',
'PART_RESPONSIBLE',
'ENG_PART_REV',
# 'ENG_REV_NO',
'FUNCTION_GROUP',
'SUB_GROUP',
'PART_LEVEL',
'SOURCE_CODE',
'VARIANT',
'MATURITY',
'INVENTORY_PART_PLANNING',
'RELEASE_STATUS',
'INVENTORY_PART_STATUS'
]

parts_df = parts_df[part_cols_ordered]

In [89]:
# # we're not going to default any values from 3dx 
# # fill blank UNIT_CODE with LTR for FLA
# parts_df['UNIT_CODE'] = np.where(((parts_df['UNIT_CODE'].isna()) & (parts_df['SOURCE_CODE'] == 'FLA')), 'LTR', parts_df['UNIT_CODE'])

# # fill remaining blank UNIT_CODE with PCS
# parts_df['UNIT_CODE'] = parts_df['UNIT_CODE'].fillna('PCS')


In [90]:
parts_df['PART_LEVEL'] = parts_df['PART_LEVEL'].astype(int)
# parts_df['ENG_PART_REV'] = np.round(parts_df['ENG_PART_REV'], decimals=2)
parts_df['WEIGHT_NET'] = np.round(parts_df['WEIGHT_NET'], decimals = 4)
# parts_df['WEIGHT_NET'] = parts_df['WEIGHT_NET'].truncate(8)


# Remove duplicates from parts_df and structure_df

We've done all the work and calculated the quantities.  IFS only need to be told about each of the parts once, and told of the structures once.
We don't sum the quantities again - we want 1 steering wheel and reference it in 4 places for the options. Just need to tell IFS once

If we have this situation, where T50-B0897 and T50-B0020 part and parent are mentioned more than once, we only need to tell IFS once:

```   
SA_Index           Part Number  Parent Part  Issue Level
17481.0_T50-B0198  T50-B0198    T50-B0020    3.0            1
17481.0_T50-B0365  T50-B0365    T50-B0020    1.0            1
17481.0_T50-B0841  T50-B0841    T50-B0020    1.0            1
17481.0_T50-B0843  T50-B0843    T50-B0020    1.0            1
17481.0_T50-B0845  T50-B0845    T50-B0020    1.0            1
17481.0_T50-B0847  T50-B0847    T50-B0020    1.0            1
>>17481.0_T50-B0897  T50-B0897    T50-B0020    1.0            1<<
17481.0_TFF-SA800  TFF-SA800    T50-B0020    1.0            1
17501.0_T50-B0198  T50-B0198    T50-B0020    3.0            1
17501.0_T50-B0365  T50-B0365    T50-B0020    1.0            1
17501.0_T50-B0841  T50-B0841    T50-B0020    1.0            1
17501.0_T50-B0843  T50-B0843    T50-B0020    1.0            1
17501.0_T50-B0845  T50-B0845    T50-B0020    1.0            1
17501.0_T50-B0847  T50-B0847    T50-B0020    1.0            1
>>17501.0_T50-B0897  T50-B0897    T50-B0020    1.0            1<<
17501.0_TFF-AA059  TFF-AA059    T50-B0020    1.0            1
```

In [91]:
parts_df = parts_df.drop_duplicates(subset=['PART_NO','ENG_PART_REV'])
# structure_df = structure_df.drop_duplicates()
# rather that drop duplicates in structures we should merge the parts and sum the qtys
# cleansed_df[['Parent Part','Title','Revision','Quantity']][cleansed_df['Title'] == 'T48E-P06323']
structure_df = structure_df.groupby(['PART_NO','PART_REV','SUB_PART_NO','SUB_PART_REV']).QTY.sum().reset_index()


In [92]:
# these structures are still duplicated
structure_df[structure_df.duplicated(subset=['PART_NO','SUB_PART_NO','SUB_PART_REV'], keep=False)].sort_values(by=['PART_NO','SUB_PART_NO'])
# structure_df[(structure_df['PART_NO'] == 'T50-A5285') & (structure_df['SUB_PART_NO'] == 'TFF-SA907')]

,PART_NO,PART_REV,SUB_PART_NO,SUB_PART_REV,QTY


# Increment ENG_PART_REV

If there is a released part already in IFS with the same ENG_PART_REV, and we're changing the structure of the part, IFS will ignore it.  We need to increment this value to make IFS recognise the change.  

In [93]:
# to stop trailing zeros when writing out to csv
parts_df['ENG_PART_REV'] = parts_df['ENG_PART_REV'].apply(str)
structure_df['PART_REV'] = structure_df['PART_REV'].apply(str)
structure_df['SUB_PART_REV'] = structure_df['SUB_PART_REV'].apply(str)

# Build a DELTA

In [94]:
if prev_timestamps != 'None':
    prev_part, prev_struct = read_prev_mig_files(outdir, prev_timestamps)

    # derive incrementer from previous part file
    # incrementer = prev_part['ENG_PART_REV'].str.split('.').str[-1].astype(int).unique()
    incrementer = prev_part['ENG_PART_REV'].astype(str).str.split('.').str[-1].astype(int).unique()
    try:
        len(incrementer) == 1
    except:
        logit.exception("More than one migration revision found in previous file")
        raise

    incrementer = incrementer[0]
    incrementer += 1

    delta_parts, delta_struct = build_delta_files(parts_df, structure_df)

    changed_parts = set(delta_parts['PART_NO'].tolist())
    # get the assembly (SA_Index) for any assembly that has those Part numbers
    delta_sa_index = cleansed_df['SA_Index'].str.split('_').str[0][cleansed_df['Part Number'].isin(changed_parts)].tolist()

    # set to give us a unique list
    sa_set = set(delta_sa_index)

    delta_df = pd.DataFrame()
    rel_delta_df = pd.DataFrame()
    unrel_delta_df = pd.DataFrame()

    # create regex pattern for word match at start of string followed by any number of chars
    if len(sa_set) > 0:
        pat = '|'.join(r"\b^{}.*\b".format(x) for x in sa_set)
        # build the delta_df
        delta_df = cleansed_df[cleansed_df['SA_Index'].str.contains(pat)]
        # sort it
        delta_df = delta_df.sort_values('orig_sort')
    else:
        logit.warning("No changes found for this delta")
        print ("No changes found for this delta")

else:
    # first time in
    incrementer = 1


Testing not incrementing the PART_REV/SUB_PART_REV

In [95]:
# this is the amount we'll add to the ENG_PART_REV to avoid issues in IFS
# make a string version of the incrementor
str_incr = '.' + str(incrementer)
# parts_df['ENG_PART_REV'] = parts_df['ENG_PART_REV'] + str_incr
# structure_df['PART_REV'] = structure_df['PART_REV'] + str_incr
# structure_df['SUB_PART_REV'] = structure_df['SUB_PART_REV'] + str_incr

# Check for Make without Buys

IFS won't handle parents of Make where there are no child parts to buy

if MAKE and there is no BUY below next row's part level less than or equal to current part level we have a MAKE without a BUY

In [96]:
if prev_timestamps != 'None':
    make_no_buy = check_make_no_buy(delta_df)

    # remove any assemblies with make and no buy
    make_no_buy_df = pd.DataFrame()
    if len(make_no_buy) > 0:
        pat = '|'.join(r"\b^{}.*\b".format(x.split('_')[0]) for x.split('_')[0] in make_no_buy)
        rel_sc_delta_df = rel_sc_delta_df[~rel_sc_delta_df['SA_Index'].str.contains(pat)]

        # get the make no buy part for writing out the warning messages
        make_no_buy_df = delta_df[delta_df['SA_Index'] == x]

    # pass all the remaining parts left in the 
    delta_parts_for_all_sa = rel_sc_delta_df['Part Number'].unique().tolist()


In [97]:
if prev_timestamps != 'None':
    
    logit.info("{} Assemblies have changed".format(len(sa_set)))
    logit.info("{} changed assemblies are not completely released so won't be processed".format(len(unrel_sa_set)))
    logit.warning("{} blank source codes that will stop the whole assembly being released".format(len(empty_sc)))
    logit.warning("{} Make parts without any Buy child parts that will stop the whole assembly being migrated".format(len(make_no_buy)))
    if unrel_delta_df.shape[0] > 0:
        for i, row in unrel_delta_df[['Function Group', 'Sub Group', 'Part Number', 'Parent Part', 'Release Status', 'SA_Index']].iterrows():
            logit.warning("Part of Unreleased assembly and not processed: {} {} {} {} {}".format(row['Function Group'], row['Sub Group'], row['Part Number'], row['Parent Part'], row['Release Status']))

    if make_no_buy_df.shape[0] > 0:
        for i, row in make_no_buy_df[['Function Group', 'Sub Group', 'Part Number', 'Parent Part', 'Source Code', 'SA_Index']].iterrows():
            logit.warning("Make Part with no buy child so assembly not processed: {} {} {} {} {}".format(row['Function Group'], row['Sub Group'], row['Part Number'], row['Parent Part'], row['Source Code']))
            
    print ("{} Assemblies have changed".format(len(sa_set)))
    print ("{} changed assemblies that are not completely released so won't be migrated".format(len(unrel_sa_set)))
    print ("{} blank source codes".format(len(empty_sc)))
    print ("{} Make parts without any Buy child parts".format(len(make_no_buy)))


In [98]:
if prev_timestamps != 'None':

    # delta_all_parts = cleansed_df['Part Number'][cleansed_df['SA_Index'].str.contains('^{}'.format(sa_set))].to_list()
    # delta_parts_df = parts_df[parts_df['PART_NO'].isin(delta_parts_for_all_sa)]
    delta_structure_df = structure_df[structure_df['PART_NO'].isin(delta_parts_for_all_sa)]

    delta_all_parts = set(delta_parts_for_all_sa + delta_structure_df['SUB_PART_NO'].tolist())

    delta_parts_df = parts_df[parts_df['PART_NO'].isin(delta_all_parts)]


# Build a test file

Provide a part number and will build test files

In [99]:
cleansed_df.orig_sort[cleansed_df['orig_sort'] == 5]

Series([], Name: orig_sort, dtype: int64)

In [100]:
# cleansed_df['SA_Index'][cleansed_df['Title'] == test_part]
cleansed_df['Title'][cleansed_df['SA_Index'] == '3467']

Series([], Name: Title, dtype: object)

In [101]:
# test_part = 'T48E-N00471'
# test_sa_index = set(cleansed_df.orig_sort[cleansed_df['Title'] == test_part])
# for x in test_sa_index:
#     test_parts = pd.DataFrame()
#     temp = []
#     temp = cleansed_df['Title'][cleansed_df['SA_Index'] == str(x)]
#     test_parts = pd.concat([test_parts, temp])

# all_test_parts = set(test_parts[0].to_list())

# test_structure_df = structure_df[structure_df['PART_NO'].isin(all_test_parts)]

# test_parts_df = parts_df[parts_df['PART_NO'].isin(all_test_parts)]

# Drop blanks

In [102]:

# there shouldn't be anything left to drop but we have to drop anything that has NA in any of the columns - must be missing data somewhere
empty_values = parts_df[parts_df[['SOURCE_CODE','UNIT_CODE','PROVIDE']].isnull().any(axis=1)]
# parts_df = parts_df[~parts_df[['SOURCE_CODE','UNIT_CODE','PROVIDE']].isnull().any(axis=1)]


In [103]:
empty_values[['SOURCE_CODE','UNIT_CODE','PROVIDE']]

,SOURCE_CODE,UNIT_CODE,PROVIDE
285,NaN,PCS,Phantom
286,NaN,PCS,Buy
287,NaN,PCS,Buy
291,NaN,PCS,Buy
292,NaN,PCS,Buy
...,...,...,...
1920,NaN,PCS,Buy
1921,NaN,PCS,Buy
1922,NaN,PCS,Buy
1923,NaN,PCS,Buy


In [104]:
def write_to_excel(df, outfile):
    with pd.ExcelWriter(os.path.join(outdir, outfile), engine="openpyxl") as writer:
            df.to_excel(writer, sheet_name = 'Sheet1', index=False)
            ws = writer.sheets['Sheet1']
            wb = writer.book
            excel_formatting.adjust_col_width_from_col(ws)


In [105]:

def write_to_csv(df, outfile):
    df.to_csv(os.path.join(outdir, outfile),sep='\t', index=False, encoding='utf-8')
    # parts_df.to_csv(os.path.join(outdir, outfile),sep='\t', index=False, encoding='utf-8')


In [106]:
# Validation checks before writing out.  Don't write files without an error_count of zero
timestr = get_current_time()

TEST=False
DELTA=False

if TEST:
    print ("*** TEST MODE ***")
    out_structure_df = test_structure_df
    out_parts_df = test_parts_df
else:
    out_structure_df = structure_df
    out_parts_df = parts_df
     

error_count = 0

# check for zero quantities
zero_quantities = out_structure_df[out_structure_df['QTY'] == 0]
if zero_quantities.shape[0] > 0:
    zero_quantities.to_excel(os.path.join(base, '{}_Zero_Quantity.xlsx'.format(project)))
    logit.error("Zero Quantities found - needs resolving first!")
    print ("Zero Quantities found - needs resolving first!")
    error_count += 1

# check for decimal quantities
decimal_quantities = out_structure_df[(out_structure_df['QTY'] > 0) & (out_structure_df['QTY'] < 1)]
if decimal_quantities.shape[0] > 0:
    decimal_quantities.to_excel(os.path.join(base, '{}_Decimal_Quantity.xlsx'.format(project)))
    logit.error("Decimal Quantities found - needs resolving first!")
    print ("Decimal Quantities found - needs resolving first!")
    error_count += 1

# check all parts present
orphaned_parts = []
orphaned_parts = pd.merge(out_parts_df, out_structure_df, left_on='PART_NO', right_on='SUB_PART_NO', how='left', indicator=True)
orphaned_parts = orphaned_parts[['PART_NO_x','SUB_PART_NO']][orphaned_parts['_merge'] == 'left_only']
if orphaned_parts.shape[0] > 1:
    logit.error("Expecting just the top CAR part to not have any parent")
    logit.error("orphaned parts {}".format(orphaned_parts))
    print ("Expecting just the top CAR part to not have any parent")
    print (orphaned_parts)
    print ("")
    error_count =+ 1

# check all sub parts in structure file are also in individual parts file
no_child_part = []
no_child_part = pd.merge(out_structure_df, out_parts_df, left_on='SUB_PART_NO', right_on='PART_NO', how='left', indicator=True)
no_child_part = no_child_part[['PART_NO_x','SUB_PART_NO']][no_child_part['_merge'] == 'left_only']
if no_child_part.shape[0] > 0:
    # not sure this check is true any more
    logit.error ("Not expecting any sub parts in structure file without individual part entry")
    logit.error (no_child_part)
    print ("Not expecting any sub parts in structure file without individual part entry")
    print (no_child_part)
    # not worried about errors at the moment as I want a part file on its own
    # error_count =+ 1

# st
sub_part_rev_check = []
sub_part_rev_check = pd.merge(out_structure_df, out_parts_df, left_on=['SUB_PART_NO','SUB_PART_REV'], right_on=['PART_NO','ENG_PART_REV'], indicator=True, how='left')
sub_part_rev_check = sub_part_rev_check[sub_part_rev_check['_merge'] == 'left_only']
if sub_part_rev_check.shape[0] > 0:
    for x, row in sub_part_rev_check.iterrows():
        logit.error ("sub_part_rev_check: SUB_PART_NO: {} SUB_PART_REV: {} ENG_PART_REV: {} is missing from structures file".format(row.iloc[2], row.iloc[4], row.iloc[1]))
        print ("sub_part_rev_check: PART_NO: {} SUB_PART_REV: {} ENG_PART_REV: {} is missing from structures file".format(row.iloc[2], row.iloc[4], row.iloc[1]))
        # print (sub_part_rev_check)
        error_count =+ 1        

# check part_rev is not blank (will be string 'nan' by now)
part_rev_check = out_structure_df[out_structure_df.PART_REV == 'nan']
if part_rev_check.shape[0] > 0:
    for x, row in part_rev_check.iterrows():
        logit.error ("part_rev_check: missing PART_REV for part {}".format(row.iloc[0], row.iloc[1], row.iloc[2]))
        # logit.error (part_rev_check)
        print ("part_rev_check: missing PART_REV for part {}".format(row.iloc[0], row.iloc[1], row.iloc[2]))
    print (part_rev_check)
    error_count =+ 1

# find master parts in out_structure_df and check PART_REV and SUB_PART_REV match - ignore top row
master_part = out_structure_df[['PART_REV','PART_NO']]
missing_master = pd.merge(master_part, out_structure_df, left_on=['PART_REV','PART_NO'], right_on=['SUB_PART_REV','SUB_PART_NO'], how='left', indicator=True)
missing_master = missing_master[missing_master['_merge'] == 'left_only']
# top part to ignore as won't be a match
top_part = out_structure_df.iloc[0]['PART_NO']
missing_master = missing_master[~missing_master['PART_NO_x'].str.contains(top_part)]
if TEST:
    # for a test file we won't need the structure for the actual assembly we've specified
    missing_master = missing_master[~missing_master['PART_NO_x'].str.contains(test_part)]

if missing_master.shape[0] > 0:
    for x, row in missing_master.iterrows():
        logit.error ("missing_parent: {} Must find PART_REV / PART_NO combo in SUB_PART_REV / SUB_PART_NO".format(row.iloc[1]))
        # logit.error (missing_master)
        print ("missing_parent: {} Must find PART_REV / PART_NO combo in SUB_PART_REV / SUB_PART_NO".format(row.iloc[1]))
    # print (missing_master)
    # not checking this at the moment - just want a file
    # error_count =+ 1   

long_pn = parts_df[parts_df['PART_NO'].str.len() > 25]
if long_pn.shape[0] > 0:
    print ("PART_NO is longer than 25 chars")
    print (long_pn['PART_NO'])
    error_count =+ 1


# all the errors relate to structure file validation.  Parts file can be produced

outfile_part = 'P_{}_{}_{}_{}'.format(project, env, incrementer, timestr)
outfile_structure = 'S_{}_{}_{}_{}'.format(project, env, incrementer, timestr)

if TEST:
    outfile_part = outfile_part + '_' + test_part
    if error_count == 0: 
        outfile_structure = outfile_structure + '_' + test_part

if DELTA:
    logit.info("*** Writing out changes since {} file ***".format(prev_timestamps))
    print ("*** Writing out changes since {} file ***".format(prev_timestamps))
    write_to_excel(delta_parts_df, 'DELTA_{}.xlsx'.format(outfile_part))
    write_to_csv(delta_parts_df, 'DELTA_{}.txt'.format(outfile_part))
    if error_count == 0: 
        write_to_excel(delta_structure_df, 'DELTA_{}.xlsx'.format(outfile_structure))
        write_to_csv(delta_structure_df, 'DELTA_{}.txt'.format(outfile_structure))

    # write out the highlighted changes
    compare_file = 'COMPARE_{}_vs_{}.xlsx'.format(prev_timestamps, timestr)
    compare_out = os.path.join(outdir, compare_file)
    df_final_parts = compare_parts(delta_parts)
    try:
        delta_struct.loc[1,]
        try:
            delta_struct.loc[2,]
            df_final_struct = compare_struct(delta_struct)
        except (KeyError):
            print ("No current parts - assumed all changes are parts being removed")
            # nothing to compare so just provide an empty delta_struct
            df_final_struct = delta_struct
    except (KeyError) as e:
        print ("No previous parts - assumed all changes are parts being added")
        df_final_struct = delta_struct
    # df_final_part.style.apply(highlight_diff, axis=None).to_excel(compare_out, engine='openpyxl',)
    with pd.ExcelWriter(compare_out) as writer:
        try:
            df_final_struct.loc[1,]
            try:
                df_final_struct.loc[2,]
                # highlight the differences between index 1 and 2
                df_final_struct.style.apply(highlight_diff, axis=None).to_excel(writer, sheet_name='Structure')
            except (KeyError) as e:
                # nothing in 2 to compare.  Write out the structures involved with the parts
                delta_structure_df.to_excel(writer, sheet_name='Structure')
        except (KeyError) as e:
            # nothing in 1 to compare.  Write out the structures involved with the parts
            delta_structure_df.to_excel(writer, sheet_name='Structure')

        df_final_parts.style.apply(highlight_diff, axis=None).to_excel(writer, sheet_name='Parts')

    # write out the delta df file for Lorena to file a bom-like file
    delta_bom_file = 'Changed_Assemblies_BOM_{}_{}_vs_{}.xlsx'.format(project, prev_timestamps, timestr)
    delta_bom_out = os.path.join(outdir, delta_bom_file)
    write_to_excel(delta_df, delta_bom_out)

# parts files can be written out - error checks are structure file only
write_to_excel(out_parts_df, outfile_part + '.xlsx')
write_to_csv(out_parts_df, outfile_part + '.txt')
if error_count == 0: 
    write_to_excel(out_structure_df, outfile_structure + '.xlsx')
    write_to_csv(out_structure_df, outfile_structure + '.txt')





Expecting just the top CAR part to not have any parent
                  PART_NO_x SUB_PART_NO
0                T53-Z00002         NaN
141               T26-A2135         NaN
338              T53-P02751         NaN
339  COPY (1) OF T53-A00823         NaN
340              T53-P05033         NaN
345              T53-A01001         NaN
696      MULTI-BRANCHABLE10         NaN
697           BRACKET IDEA3         NaN
698           BRACKET IDEA2         NaN
699      MULTI-BRANCHABLE13         NaN
700  HPS40-2 BUCHSENSTECKER         NaN

missing_parent: T53-A01001 Must find PART_REV / PART_NO combo in SUB_PART_REV / SUB_PART_NO
missing_parent: T53-A01001 Must find PART_REV / PART_NO combo in SUB_PART_REV / SUB_PART_NO
missing_parent: T53-A01001 Must find PART_REV / PART_NO combo in SUB_PART_REV / SUB_PART_NO
missing_parent: T53-A01001 Must find PART_REV / PART_NO combo in SUB_PART_REV / SUB_PART_NO
missing_parent: T53-A01001 Must find PART_REV / PART_NO combo in SUB_PART_REV / SUB_PART_NO
miss

In [216]:
# existing_bom[existing_bom['Title'] == 'T48e-P06323']
out_parts_df[out_parts_df['PART_NO'] == 'T48E-P06323']
# out_structure_df[out_structure_df['SUB_PART_NO'] == 'T48E-P06323']

,PART_NO,DESCRIPTION,WEIGHT_NET,INFO_TEXT,UNIT_CODE,LOT_TRACKING_CODE,SERIAL_RULE,SERIAL_TRACKING_CODE,CONFIGURABLE,PROVIDE,...,ENG_PART_REV,FUNCTION_GROUP,SUB_GROUP,PART_LEVEL,SOURCE_CODE,VARIANT,MATURITY,INVENTORY_PART_PLANNING,RELEASE_STATUS,INVENTORY_PART_STATUS


In [217]:
pd.merge(out_structure_df, out_parts_df, left_on=['SUB_PART_NO','SUB_PART_REV'], right_on=['PART_NO','ENG_PART_REV'], indicator=True, how='left')
# sub_part_rev_check
# out_parts_df.filter(regex='PART|REV')
out_structure_df[out_structure_df['SUB_PART_NO'] == 'T48E-P06323']
existing_bom[existing_bom['Title'] == 'T48e-P06323']

,orig_sort,BOM COUNT,Matching Key,Last Export Date,Function Group,System,Sub System,Level,Title,Parent Part,...,Target Mass,Type,Variant/Option,extr_function,extr_invalid_code,extr_maturity,extr_pn,extr_project,has_child,part_number_length


In [218]:
print (structure_df[structure_df['PART_NO'] == 'T48E-A02195'])
print (parts_df[parts_df['PART_NO'] == 'T48e-A02195'])
existing_bom[['Title','Source Code','UOM','Provide','Inventory Part Planning']][existing_bom['Title'] == 'T48e-A02195']
cleansed_df[['Title','Source Code','UOM','Provide','Inventory Part Planning']][cleansed_df['Title'] == 'T48e-A02195']

Empty DataFrame
Columns: [PART_NO, PART_REV, SUB_PART_NO, SUB_PART_REV, QTY]
Index: []
Empty DataFrame
Columns: [PART_NO, DESCRIPTION, WEIGHT_NET, INFO_TEXT, UNIT_CODE, LOT_TRACKING_CODE, SERIAL_RULE, SERIAL_TRACKING_CODE, CONFIGURABLE, PROVIDE, AQUISITION_CODE, PLANNING_METHOD, PART_RESPONSIBLE, ENG_PART_REV, FUNCTION_GROUP, SUB_GROUP, PART_LEVEL, SOURCE_CODE, VARIANT, MATURITY, INVENTORY_PART_PLANNING, RELEASE_STATUS, INVENTORY_PART_STATUS]
Index: []

[0 rows x 23 columns]


,Title,Source Code,UOM,Provide,Inventory Part Planning


In [219]:
# pd.merge(structure_df, parts_df, left_on='SUB_PART_NO', right_on='PART_NO', how='left', indicator=True)
cleansed_df.filter(regex='Title|Source|Part')[cleansed_df['Parent Part'] == 'T48E-A02773X']

,Title,Source Code,Inventory Part Planning,Part Type,Parent Part


In [220]:
part_rev_check

,PART_NO,PART_REV,SUB_PART_NO,SUB_PART_REV,QTY


In [221]:
existing_bom.filter(regex='Title|Level|Source|SA_Index').loc[50:57]

,Level,Title,Source Code,Instance Title,SA_Index,Source Code Name,Surface Maturity Level
50,8.0,T53-A01005,NaN,T53-A01005.1,3_4,NaN,NaN
51,9.0,T53-A00905,NaN,T53-A00905.1,3_4,NaN,NaN
52,10.0,T53-A00879,NaN,T53-A00879.1,3_4,NaN,NaN
53,10.0,T53-A01463,NaN,T53-A01439.1,3_4,NaN,NaN
54,11.0,T53-A01439,BOF,T53-A01439.1,3_4,BOF - Bought Out Finished,NaN
55,11.0,TFF-SK202,NaN,TFF-SK202.1,3_4,NaN,NaN
56,10.0,T53-A01129,BOF,T53-A01129.1,3_4,NaN,NaN
57,10.0,T53-A01131,BOF,T53-A01131.1,3_4,NaN,NaN


In [222]:
logit.info('Completed')

for handler in logit.handlers:
    if isinstance(handler, logging.FileHandler):
        handler.close()

In [223]:
for handler in logit.handlers:
    if isinstance(handler, logging.FileHandler):
        handler.close()